In [6]:
import rasterio
import numpy as np
import joblib
from joblib import load
import sklearn
import pandas as pd

In [7]:
rf_model = joblib.load("rf_model_gedi_sentinel.joblib")

with rasterio.open("merge_s1_s2.tif") as merge:
    merge_dt = merge.read()  # Shape: (bands, height, width)
    profile = merge.profile  # Save the metadata for later
    transform = merge.transform

In [8]:
expected_feature_names = rf_model.feature_names_in_
print(expected_feature_names)

['B2' 'B3' 'B4' 'B5' 'B6' 'B7' 'B8' 'B11' 'B12' 'VV' 'VH']


In [9]:
merge_data = np.transpose(merge_dt, (1, 2, 0))
pixels = merge_data.shape[0] * merge_data.shape[1]
bands = merge_data.shape[2]
merge_data_2d = merge_data.reshape(pixels, bands)
feature_names = ["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B11", "B12", "VV", "VH"]
merge_data_df = pd.DataFrame(merge_data_2d, columns=feature_names)

# Apply the model to predict
predictions = rf_model.predict(merge_data_df)

predictions_raster = predictions.reshape(merge_data.shape[0], merge_data.shape[1])

output_profile = profile
output_profile.update(dtype=rasterio.float32, count=1)

with rasterio.open("predictions_rf_agb.tif", "w", **output_profile) as dest:
    dest.write(predictions_raster.astype(rasterio.float32), 1)